# NLP From Scratch: Translation with a Sequence to Sequence Network and Attention

In this project we will be teaching a neural network to translate from German to English.


This is made possible by the simple but powerful idea of the [sequence
to sequence network](https://arxiv.org/abs/1409.3215), in which two
recurrent neural networks work together to transform one sequence to
another. An encoder network condenses an input sequence into a vector,
and a decoder network unfolds that vector into a new sequence.


To improve upon this model we'll use an [attention
mechanism](https://arxiv.org/abs/1409.0473), which lets the decoder
learn to focus over a specific range of the input sequence.

## Dependencies

In [4]:
# using Python 3.9
%%capture
%pip install pandas torch matplotlib numpy ipython

UsageError: Line magic function `%%capture` not found.


In [2]:
from __future__ import unicode_literals, print_function, division
import os
from io import open
import unicodedata
import string
import re
import random
import time
import math
import pandas as pd

from IPython.display import Image
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Loading & preprocessing training data

The files are all in Unicode, to simplify we will turn Unicode
characters to ASCII, make everything lowercase, and trim most
punctuation. We then implement a parsing funciton that reads a csv file from the specified path, normalizes every sentence in the dataset using the normalize function, and returns a list of (german_sentence, english_sentence) pairs. If the parameter reverse is True, the order of languages in the tuple are reversed resulting in reverse translation.



In [5]:
# https://stackoverflow.com/a/518232/2809427
def unicode_to_ascii(s):
    return ''.join(
        'ss' if c == 'ß' else c # account for eszett
        for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# lowercase, trim, and remove non-letter characters
def normalize_string(s):
    s = unicode_to_ascii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s


def parse_data(path, reverse=False):

  pairs = pd.read_csv(path)
  pairs['ENG'] = pairs['ENG'].apply(normalize_string)
  pairs['GER'] = pairs['GER'].apply(normalize_string)

  if reverse:
    pairs = pairs[['GER', 'ENG']]

  return pairs

Since there are a *lot* of example sentences and we want to train
something quickly, we'll trim the data set to only relatively short and
simple sentences and we will avoid explicit questions. Hence we will set a maximum sentence length of 10 words (that includes punctuation) and remove questions.



In [6]:
MAX_WORDS = 10

interrogative_words = ( "what", "why", "how", "which", "where", "when" , "who", "whose",
                        "was", "warum", "wie", "welche", "wo", "wann", "wer", "wieso")

"""
TODO: Task 1 (10 pt)

Implement the filter_pairs function so that it takes in a pd.DataFrame of language pairs and outputs another dataframe where following pairs are removed:

- those where one of both exceeds the maximal sentence length (MAX_WORDS)
- those where the pair contains any of the words defined in interrogative_words or a question mark.

Note: your method should NOT employ any explicit iteration (i.e. for loops).

"""

def filter_pairs(pairs,
                 drop_words=interrogative_words,
                 max_words=MAX_WORDS):

    pairs = pairs[
        (pairs['ENG'].str.split().str.len() < max_words) & # account for zero indexing
        (pairs['GER'].str.split().str.len() < max_words)
    ]

    # creating a regex pattern for interrogative words and question marks
    question_pattern = fr"\b(?:{'|'.join(drop_words)})\b|\?"

    # filter based on interrogative words and question marks
    pairs = pairs[
        ~pairs['ENG'].str.contains(question_pattern, case=False, regex=True) &
        ~pairs['GER'].str.contains(question_pattern, case=False, regex=True)
    ]

    return pairs

Each word in a language will be represented as a one-hot
vector, or giant vector of zeros except for a single one (at the index
of the word). Compared to the dozens of characters that might exist in a
language, there are many many more words, so the encoding vector is much
larger. We will however cheat a bit and trim the data to only use a few
thousand words per language.

We'll need a unique index per word to use as the inputs and targets of
the networks later. To keep track of all this we will use a helper class
called ``Language`` which has word → index (``word2index``) and index → word
(``index2word``) dictionaries, as well as a count of each word
``word2count`` which will be used to replace rare words later.




In [7]:
SOS_token = 0
EOS_token = 1

"""
TODO: Task 2 (10 pt)

Implement the function remove_rare_words, that removes words whose frequency is below
min_freq and makes sure that the size of the vocabulary is below max_vocab_size
by iteratively removing least frequent words.

The function should also update the indices in the dictionary, so that no index
is left unused (i.e. all values from 0 to n_words are indices) and all variables are up-to-date.

You might want to define a helper method remove_word to break down the task.

"""


class Language:
    def __init__(self):
        self.word2index = {} # maps word to integer index
        self.word2count = {} # maps word to its frequency
        self.index2word = {0: "SOS", 1: "EOS"} # maps index to a word
        self.n_words = 2  # Count SOS and EOS

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

    def remove_word(self, word):
      if word in self.word2index:
        index = self.word2index[word]
        del self.index2word[index] # remove word from integer to word index
        del self.word2index[word] # remove word from word to integer index
        del self.word2count[word] # remove word from frequency count

        # update word to integer index for indices that come after removed word
        for w, i in list(self.word2index.items()):
            if i > index:
                self.word2index[w] -= 1

        # update integer to word index for words that come after removed word
        for i in range(index + 1, self.n_words):
            self.index2word[i - 1] = self.index2word[i]

        # remove the last entry of integer to word index as it is now a duplicate
        del self.index2word[self.n_words - 1]

        # update number of words
        self.n_words -= 1

    def remove_rare_words(self, min_freq, max_vocab_size):
      words_to_remove = [word for word in self.word2count if self.word2count[word] < min_freq] # list of words to remove
      words_to_remove.sort(key=lambda word: self.word2count[word]) # sort words by frequency

      # remove words until the vocabulary size is below max_vocab_size
      while self.n_words > max_vocab_size:
        if not words_to_remove:
          break
        word_to_remove = words_to_remove.pop(0)
        self.remove_word(word_to_remove)

The full process for preparing the data is:

-  Read text file and split into lines, split lines into pairs
-  Normalize text, filter by length and content
-  Make word lists from sentences in pairs




In [8]:
def prepare_data(pairs, reverse=False):

    print(f"Read {len(pairs)} sentence pairs")
    pairs = filter_pairs(pairs)
    print(f"Filtered {len(pairs)} sentence pairs")
    pairs = pairs.to_numpy()

    input_lang = Language()
    output_lang = Language()

    for pair in pairs:
        input_lang.add_sentence(pair[0])
        output_lang.add_sentence(pair[1])

    print(f"Input language: {input_lang.n_words} words")
    print(f"Output language: {output_lang.n_words} words")
    return input_lang, output_lang, pairs

In [9]:
#path = os.path.join(os.getcwd(), "data", "translations.csv")
path = "https://raw.githubusercontent.com/smkerr/problem-set-3-ps3_group_b/main/data/translations.csv"
pairs = parse_data(path)
input_lang, output_lang, pairs = prepare_data(pairs, reverse=True)

Read 255817 sentence pairs
Filtered 145581 sentence pairs
Input language: 12982 words
Output language: 25355 words


## Encoder-decoders

A Recurrent Neural Network, or RNN, is a network that operates on a
sequence and uses its own output as input for subsequent steps.

A [Sequence to Sequence network](https://arxiv.org/abs/1409.3215)_, or
seq2seq network, or [Encoder Decoder
network](https://arxiv.org/pdf/1406.1078v3.pdf)_, is a model
consisting of two RNNs called the encoder and decoder. The encoder reads
an input sequence and outputs a single vector, and the decoder reads
that vector to produce an output sequence.

Unlike sequence prediction with a single RNN, where every input
corresponds to an output, the seq2seq model frees us from sequence
length and order, which makes it ideal for translation between two
languages, because there usally is not a 1:1 mapping between words.


### The Encoder

The encoder of a seq2seq network is a RNN that outputs some value for
every word from the input sentence. For every input word the encoder
outputs a vector and a hidden state, and uses the hidden state for the
next input word.

In [ ]:
Image(filename="img/encoder.png", height=400)

In [44]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, rnn_type='GRU', batch_size=1, device=device):
        super().__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.rnn_type = rnn_type
        self.device = device

        self.embedding = nn.Embedding(input_size, hidden_size) #produces a list of vect
        self.rnn = getattr(nn, rnn_type)(self.hidden_size, self.hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1,1,-1) # translates input (series of indices for the respective words) into their vector representation
        output, hidden = self.rnn(embedded, hidden)
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, self.batch_size, self.hidden_size, device=self.device)

Although all encoders must have certain distinctive features to be called as such, as for example the ability to process sequences of different length returning fixed-length hidden states, it is not required for them to have a recurrent structure. In fact, most state-of-the art NLP models use transformer layers, combining attention layers with feed-forward network for a higher parallelizability. Because transformer layers process information in parallel, they have no inherent mechanisms that makes them aware of the position of the word in the sentence and need to engineer a feature that encodes that information.

In [45]:
"""
TODO: Task 3 (10 pt)

Implement a positional encoding module for an alternative encoder using transformers.
The positional encoder should create a vector of size max_length containing the
discrete values of a function f(x) applied on the even features of the input
and g(x) applied on the odd features of the input.
f(x) should be defaulted to sine, g(x) to cosine.
Their value should be scaled by div_term.

Briefly describe (3-4 sentences):

- why sinusoidal are used by default
- how div_term impacts the encoding
- which characteristics would make a function a suitable
  alternative to sinusoidals.

Note: you are not expected to train the TransformerEncoder (this will most likely
not work due to incompatibility with the train method). Simply show you can initialize
the encoder and pass a tensor through it.

"""


class PositionalEncoding(nn.Module):

    def __init__(self,
                 hidden_size,
                 max_length,
                 even_fc=torch.sin,
                 odd_fc=torch.cos):

        super().__init__()
        div_term = 1 / (torch.exp(torch.arange(0, hidden_size, 2, dtype=torch.float) *
                             (math.log(10000.0) / hidden_size)))

        # positional encodings
        self.positional_encodings = torch.zeros(max_length, hidden_size) # max_length x hidden_size ensure that the positional encodings have the same size as the input
        position = torch.arange(0, max_length, dtype=torch.float).unsqueeze(1) # max_length x 1 vector containing the position of each word in the input
        self.positional_encodings[:, 0::2] = even_fc(position * div_term) # apply even function to even features in order to create the positional encodings
        self.positional_encodings[:, 1::2] = odd_fc(position * div_term) # apply odd function to odd features in order to create the positional encodings

    def forward(self, input):
        """In the forward step, the positional_encoder vector should be added to the input."""
        return input + self.positional_encodings

class TransformerEncoder(nn.Module):

    def __init__(self, input_size, hidden_size, max_length=MAX_WORDS, batch_size=1):

        super().__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.positional_encoding = PositionalEncoding(hidden_size, max_length)
        self.transformer_layers = nn.TransformerEncoderLayer(hidden_size, nhead=4)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_layers, num_layers=6)
        self.fc = nn.Linear(hidden_size, hidden_size)

    def forward(self, input, positional_encoding=False):

        embedded = self.embedding(input) # translates input (series of indices for the respective words) into their vector representation

        # If true, applies positional encoding
        if positional_encoding:
            embedded = self.positional_encoding(embedded).view(1, 1, -1)

        # Transformer encoder layers
        transformer_output = self.transformer_encoder(embedded)

        # Fully connected layer
        output = self.fc(transformer_output)

        return output

    # defining tensor from sentence method to make the encoder compatible with the decoder
    def tensorFromSentence(self, sentence):
        indexes = [input_lang.word2index[word] for word in sentence.split(' ')]
        indexes.append(EOS_token)
        return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

    def init_hidden(self):
        return torch.zeros(1, self.batch_size, self.hidden_size, device=device)

In [46]:
attn_encoder = TransformerEncoder(input_lang.n_words, 256)
input = attn_encoder.tensorFromSentence(pairs[0][0])
attn_encoder(input)
### SHOW NOTEBOOK OUTPUT ###

tensor([[[-4.1769e-01,  5.0893e-01, -9.5441e-02,  2.0867e-01, -5.9357e-01,
           5.2481e-02, -2.1749e-01, -9.2590e-02, -6.3563e-01,  5.6829e-01,
           6.2869e-01,  2.1348e-01, -1.7159e-01, -1.2895e+00, -5.7395e-01,
           6.7668e-01, -6.5501e-01, -9.0820e-01, -7.2859e-01,  2.7913e-01,
           8.4095e-01, -5.2859e-01,  3.9516e-01, -3.5808e-01, -8.4342e-01,
          -2.3671e-01, -5.7909e-02, -9.9490e-01, -9.8250e-01,  4.1512e-02,
          -5.7250e-01, -2.0012e-01,  3.0208e-01, -8.3089e-01, -1.4534e-01,
          -1.4406e-01, -8.1816e-01, -3.3774e-01,  2.6550e-01, -1.0127e-01,
          -6.6079e-01,  5.6895e-01,  8.8999e-01,  1.8085e-02, -8.0678e-01,
          -1.1518e-02,  1.0765e-01,  8.4229e-01, -2.8150e-01, -5.1609e-01,
          -4.0990e-02, -1.3272e+00, -2.7997e-01,  7.4448e-01, -1.1613e+00,
          -5.5500e-01, -8.9731e-02, -1.6422e-01,  8.5494e-01, -7.8579e-01,
           1.9517e-01,  4.8700e-02,  1.0378e-01, -7.2315e-01,  1.3632e-01,
          -3.6874e-01,  5

### The Decoder

The decoder is another RNN that takes the encoder output vector(s) and
outputs a sequence of words to create the translation.




#### Attention Decoder


Attention allows the decoder network to "focus" on a different part of the encoder's outputs for every step of the decoder's own outputs.
For this, we determine a set of attention weights which capture which part of the sentence is most important for the translation.
The attention weights will be multiplied by the encoder output vectors to create a weighted combination.
The result should contain information about that specific part of the input sequence, and thus help the decoder choose the right output words.

Specifically, this is done as follows:
1. Attention weights are calculated with a feed-forward layer (`self.attn`) using the concatenated decoder's input and hidden state as inputs and appying a softmax to it.
   * Because there are sentences of all sizes in the training data, to actually create and train this layer we have to choose a maximum sentence length (input length, for encoder outputs) that it can apply to. Sentences of the maximum length will use all the attention weights, while shorter sentences will only use the first few.
1. The attention weights will be multiplied by the encoder output vectors to create a weighted combination (using `torch.bmm()`).
1. The encoder outputs weighted by the attention weights are concatenated (use `torch.cat`) with the decoder's embeddings and fed into another feed-forward layer (`self.attn_combine`).
1. ReLu is applied on the output of the attention module.



In [ ]:
Image(filename="img/decoder.png", height=900)

In [47]:
"""
TODO: Task 4 (10 pt)
Implement the attention layer as described above and depicted in the figure.

Show the train progress (3-5 min) when using the decoder with attention on the translation task (see "Pre-trained GRU seq2seq model" section below).

"""

class Attention(nn.Module):

    def __init__(self, hidden_size, max_length):

        super().__init__()
        self.hidden_size = hidden_size
        self.max_length = max_length

        # attention layer (feed-forward layer)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        # the next feed-forward layer combines the attention weights with the encoder outputs
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)

    def forward(self, embedded, hidden, encoder_outputs):

        # calculate attention weights with self.attn using the concatenated decoder's input and hidden state as inputs and applying a softmax to it
        attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)

        # multiply attention weights with encoder outputs to get new "weighted sum" context vector (using torch.bmm())
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))

        # the encoder outputs weighted by the attention weights are concatenated (use torch.cat) with the decoder's embeddings and fed into another feed-forward layer (self.attn_combine)
        output_enc = torch.cat((embedded[0], attn_applied[0]), 1)

        # ReLu is applied on the output of the attention module
        output = F.relu(self.attn_combine(output_enc).unsqueeze(0))

        return output, attn_weights


In [48]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, rnn_type='GRU', rnn_n_layers=1, batch_size=1, max_length=MAX_WORDS+1, device=device):
        super().__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        self.rnn_type = rnn_type
        self.rnn_n_layers = rnn_n_layers
        self.batch_size = batch_size
        self.device = device

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)  # Adjust layer name to 'attn'
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)  # Adjust layer name to 'attn_combine'
        self.rnn = getattr(nn, self.rnn_type)(self.hidden_size, self.hidden_size, self.rnn_n_layers)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        c0 = self.init_hidden()
        # h0 = self.init_hidden() # reuse hidden state of LSTM/GRU cell or reinitialize?

        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        if self.rnn_type == 'LSTM':
            attn_hidden = hidden[0] # only hidden, not cell state
        else:
            attn_hidden = hidden # GRU has no cell state

        attn_weights = F.softmax(self.attn(torch.cat((embedded[0], attn_hidden[0]), 1)))
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output, hidden = self.rnn(output, hidden)
        output = self.out(output[0])
        output = F.log_softmax(output, dim=1)

        return output, hidden

    def init_hidden(self):
        weights = torch.zeros(self.rnn_n_layers, self.batch_size, self.hidden_size, device=device)
        return (weights, weights) if self.rnn_type == 'LSTM' else weights



# Training

### Preparing Training Data

To train, for each pair we will need an input tensor (indexes of the
words in the input sentence) and target tensor (indexes of the words in
the target sentence). While creating these vectors we will append the
EOS token to both sequences.




In [49]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

### Training the Model

To train we run the input sentence through the encoder, and keep track
of every output and the latest hidden state. Then the decoder is given
the `<SOS>` token as its first input, and the last hidden state of the
encoder as its first hidden state.

"Teacher forcing" is the concept of using the real target outputs as
each next input, instead of using the decoder's guess as the next input.
Using teacher forcing causes it to converge faster but [when the trained
network is exploited, it may exhibit
instability](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.378.4095&rep=rep1&type=pdf).

You can observe outputs of teacher-forced networks that read with
coherent grammar but wander far from the correct translation -
intuitively it has learned to represent the output grammar and can "pick
up" the meaning once the teacher tells it the first few words, but it
has not properly learned how to create the sentence from the translation
in the first place.

Because of the freedom PyTorch's autograd gives us, we can randomly
choose to use teacher forcing or not with a simple if statement. Turn
``teacher_forcing_ratio`` up to use more of it.




In [50]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_WORDS+1):
    encoder_hidden = encoder.init_hidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

This is a helper function to print time elapsed and estimated time
remaining given the current time and progress %.




In [51]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

### Plotting results

Plotting is done with matplotlib, using the array of loss values
``plot_losses`` saved while training.




In [52]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    return fig

The whole training process looks like this:

-  Start a timer
-  Initialize optimizers and criterion
-  Create set of training pairs
-  Start empty losses array for plotting

Then we call ``train`` many times and occasionally print the progress (%
of examples, time so far, estimated time) and average loss.




In [53]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

## Evaluation

Evaluation is mostly the same as training, but there are no targets so
we simply feed the decoder's predictions back to itself for each step.
Every time it predicts a word we add it to the output string, and if it
predicts the EOS token we stop there. We also store the decoder's
attention outputs for display later.




In [54]:
def evaluate(encoder, decoder, sentence, max_length=MAX_WORDS+1):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.init_hidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

We can evaluate random sentences from the training set and print out the
input, target, and output to make some subjective quality judgements:




In [55]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

## Training and Evaluating

With all these helper functions in place (it looks like extra work, but
it makes it easier to run multiple experiments) we can actually
initialize a network and start training.

Remember that the input sentences were heavily filtered. For this small
dataset we can use relatively small networks of 256 hidden nodes and a
single GRU layer. When training from scratch after about 40 minutes on a MacBook CPU we'll get some
reasonable results.

To speed-up the training, we do not randomly initialize the weights, but reuse the weights from a pretrained model.

Note: run this cells to show that your code is correctly functioning.



In [58]:
hidden_size = 256
encoder = EncoderRNN(input_lang.n_words, hidden_size, rnn_type='GRU').to(device)
attn_decoder = AttnDecoderRNN(hidden_size, output_lang.n_words, rnn_type='GRU', batch_size=1, dropout_p=0.1).to(device)

In [59]:
trainIters(encoder, attn_decoder, 10**3, print_every=100)
### SHOW NOTEBOOK OUTPUT ###

/var/folders/38/sb94bl795bv7klhlb8l7pj_00000gn/T/ipykernel_43001/3712431664.py:31: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  attn_weights = F.softmax(self.attn(torch.cat((embedded[0], attn_hidden[0]), 1)))


0m 5s (- 0m 49s) (100 10%) 6.5359
0m 11s (- 0m 45s) (200 20%) 5.6653
0m 17s (- 0m 40s) (300 30%) 5.4375
0m 23s (- 0m 35s) (400 40%) 5.4725
0m 29s (- 0m 29s) (500 50%) 5.1284
0m 35s (- 0m 23s) (600 60%) 5.3260
0m 41s (- 0m 17s) (700 70%) 5.3801
0m 48s (- 0m 12s) (800 80%) 5.1263
0m 54s (- 0m 6s) (900 90%) 5.0754
1m 1s (- 0m 0s) (1000 100%) 5.0443


In [60]:
evaluateRandomly(encoder, attn_decoder)
### SHOW NOTEBOOK OUTPUT ###

> you have to do this now .
= du musst das jetzt machen .
< er ist nicht ist nicht nicht . <EOS>

> i study for hours every day .
= ich studiere jeden tag stunden lang .
< ich habe nicht nicht nicht . <EOS>

> tom has to stop doing that right now .
= tom muss sofort damit aufhoren .
< tom ist nicht nicht nicht . <EOS>

> tom likes it in boston .
= tom gefallt es in boston .
< tom ist nicht nicht nicht . <EOS>

> thanks for coming .
= danke fur s kommen .
< er ist nicht nicht zu . <EOS>

> i kissed my wife .
= ich kusste meine frau .
< ich habe nicht nicht . <EOS>

> tom can t win .
= tom kann nicht gewinnen .
< tom ist nicht nicht nicht . <EOS>

> tom blames mary for everything .
= tom gibt mary fur alles die schuld .
< tom ist nicht nicht nicht . <EOS>

> the student raised his hand .
= der student hob die hand .
< er ist nicht ist nicht nicht . <EOS>

> we seldom go to boston anymore .
= wir gehen nur noch selten nach boston .
< er ist nicht nicht viel . <EOS>



/var/folders/38/sb94bl795bv7klhlb8l7pj_00000gn/T/ipykernel_43001/3712431664.py:31: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  attn_weights = F.softmax(self.attn(torch.cat((embedded[0], attn_hidden[0]), 1)))


#### Pre-trained GRU seq2seq model

Load the state dict including the weights of a pre-trained model. If the initialization does not work, the architecture of your decoder is likely incorrect. Keep in mind that the two feed-forward layers of the Attention module have to be named `attn` and `attn_combine`.

In [61]:
hidden_size = 256
encoder_file = os.path.join('data', 'pretrained_encoder2.pt')
pretrained_encoder = EncoderRNN(input_lang.n_words, hidden_size, rnn_type='GRU').to(device)
# pretrained_encoder.load_state_dict(torch.load(encoder_file), strict=False)
# pretrained_encoder.eval()
# the two commented lines above are not working because the pretrained encoder has a different vocabulary size than the one we are using; were part of the initial code

desired_vocab_size = 12982
if pretrained_encoder.embedding.weight.shape[0] != desired_vocab_size:
    pretrained_encoder.embedding = nn.Embedding(desired_vocab_size, hidden_size)

my_encoder = EncoderRNN(input_lang.n_words, hidden_size, rnn_type='GRU').to(device)
my_encoder.load_state_dict(pretrained_encoder.state_dict(), strict=False)
my_encoder.eval()
### SHOW NOTEBOOK OUTPUT ###

EncoderRNN(
  (embedding): Embedding(12982, 256)
  (rnn): GRU(256, 256)
)

In [62]:
hidden_size = 256
decoder_file = os.path.join('data', 'pretrained_decoder2.pt')
pretrained_attn_decoder = AttnDecoderRNN(hidden_size, output_lang.n_words, rnn_type='GRU', dropout_p=0.1).to(device)
# pretrained_attn_decoder.load_state_dict(torch.load(decoder_file), strict=False)
# pretrained_attn_decoder.eval()
# the two commented lines above are not working because the pretrained decoder has a different vocabulary size than the one we are using; were part of the initial code

desired_vocab_size = 25355
if pretrained_attn_decoder.embedding.weight.shape[0] != desired_vocab_size:
    pretrained_attn_decoder.embedding = nn.Embedding(desired_vocab_size, hidden_size)

my_attn_decoder = AttnDecoderRNN(hidden_size, output_lang.n_words, rnn_type='GRU', dropout_p=0.1).to(device)
my_attn_decoder.load_state_dict(pretrained_attn_decoder.state_dict(), strict=False)
my_attn_decoder.eval()

### SHOW NOTEBOOK OUTPUT ###

AttnDecoderRNN(
  (embedding): Embedding(25355, 256)
  (dropout): Dropout(p=0.1, inplace=False)
  (attn): Linear(in_features=512, out_features=11, bias=True)
  (attn_combine): Linear(in_features=512, out_features=256, bias=True)
  (rnn): GRU(256, 256)
  (out): Linear(in_features=256, out_features=25355, bias=True)
)

In [63]:
trainIters(pretrained_encoder, pretrained_attn_decoder, 10**3, print_every=100) # to be changed to 10**4 as provided initially; changed to 10**3 to reduce the time needed to run the code
### SHOW NOTEBOOK OUTPUT ###

/var/folders/38/sb94bl795bv7klhlb8l7pj_00000gn/T/ipykernel_43001/3712431664.py:31: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  attn_weights = F.softmax(self.attn(torch.cat((embedded[0], attn_hidden[0]), 1)))


0m 5s (- 0m 52s) (100 10%) 6.7667
0m 11s (- 0m 47s) (200 20%) 5.3738
0m 19s (- 0m 44s) (300 30%) 5.6792
0m 28s (- 0m 42s) (400 40%) 5.4303
0m 35s (- 0m 35s) (500 50%) 5.2713
0m 41s (- 0m 27s) (600 60%) 5.3032
0m 48s (- 0m 20s) (700 70%) 5.5730
0m 54s (- 0m 13s) (800 80%) 5.3596
1m 1s (- 0m 6s) (900 90%) 5.4227
1m 7s (- 0m 0s) (1000 100%) 5.2920


In [64]:
evaluateRandomly(pretrained_encoder, pretrained_attn_decoder)
### SHOW NOTEBOOK OUTPUT ###

> i may still win .
= vielleicht gewinne ich noch .
< ich bin nicht . . <EOS>

> tom owes his job to mary .
= tom verdankt seine arbeit maria .
< ich bin nicht tom . . <EOS>

> the train is leaving soon .
= der zug fahrt bald ab .
< ich bin nicht . . . <EOS>

> tom found a good job .
= tom hat eine gute arbeit gefunden .
< wir sind nicht . . <EOS>

> i m very proud of it .
= ich bin darauf sehr stolz .
< ich bin nicht . . . <EOS>

> i m glad you re still here .
= ich freue mich dass du noch hier bist .
< ich bin nicht tom . . <EOS>

> i hope you re not serious .
= ich hoffe du meinst das nicht ernst .
< ich bin nicht . . <EOS>

> she stopped picking daisies .
= sie horte auf ganseblumchen zu pflucken .
< ich bin nicht . . <EOS>

> i m restless .
= ich bin ruhelos .
< ich bin nicht . <EOS>

> i thought that you d be ready .
= ich dachte du warest so weit .
< ich bin nicht tom . . <EOS>



/var/folders/38/sb94bl795bv7klhlb8l7pj_00000gn/T/ipykernel_43001/3712431664.py:31: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  attn_weights = F.softmax(self.attn(torch.cat((embedded[0], attn_hidden[0]), 1)))


# Transfer learning
## Fine tune out-of-the box encoder-decoder model

In [10]:
"""
TODO: Task 5 (10 pt)

Load a transformer for the hugging face library and fine-tune it to your problem.
Show the training progress of the model and the training metrics. Briefly
explain (3-4 sentences) the model choice and comment on the outcomes.

Note: you don't have to use the above-defined methods for training.
Splitting the dataset in train and test is welcomed, but not required for the task.

"""
from transformers import MarianMTModel, MarianTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader

In [11]:
model_name = 'Helsinki-NLP/opus-mt-en-de'
model = MarianMTModel.from_pretrained(model_name)
tokenizer = MarianTokenizer.from_pretrained(model_name)

In [28]:
source_text = [pair[0] for pair in pairs]
tokens = tokenizer(source_text, return_tensors="pt", padding=True)

In [29]:
labels = tokenizer([pair[1] for pair in pairs], return_tensors="pt", padding=True)["input_ids"]

In [30]:
train_dataset = torch.utils.data.TensorDataset(tokens["input_ids"], tokens["attention_mask"], labels)
# smaller sample size
train_dataset = torch.utils.data.Subset(train_dataset, range(0, 5000))
train_loader = DataLoader(train_dataset, batch_size=4)
val_dataset = torch.utils.data.Subset(train_dataset, range(len(train_dataset) - 500, len(train_dataset)))
val_loader = DataLoader(val_dataset, batch_size=4)
len(train_dataset), len(val_dataset)

(5000, 500)

In [31]:
num_epochs = 5
learning_rate = 5e-5
accumulation_steps = 4
optimizer = AdamW(model.parameters(), lr=learning_rate)
effective_training_steps = len(train_loader) * num_epochs // accumulation_steps
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=effective_training_steps)

for epoch in range(num_epochs):
    running_loss = 0.0

    for i, batch in enumerate(train_loader, 1):
        # Unpack the batch
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass
        loss.backward()

        if i % accumulation_steps == 0 or i == len(train_loader):
            # Update model parameters after accumulating gradients
            optimizer.step()
            optimizer.zero_grad()

        running_loss += loss.item()

        if i % 100 == 0:  # Print every 100 batches
            print(f"Epoch: {epoch + 1}, Batch: {i}/{len(train_loader)}, Loss: {loss.item():.4f}")

    average_loss = running_loss / len(train_loader) / accumulation_steps

    print(f"Epoch: {epoch + 1}, Average Loss: {average_loss:.4f}")

/opt/homebrew/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch: 1, Batch: 100/1250, Loss: 0.0924
Epoch: 1, Batch: 200/1250, Loss: 0.1190
Epoch: 1, Batch: 300/1250, Loss: 0.8114
Epoch: 1, Batch: 400/1250, Loss: 0.6717
Epoch: 1, Batch: 500/1250, Loss: 1.2040
Epoch: 1, Batch: 600/1250, Loss: 0.3516
Epoch: 1, Batch: 700/1250, Loss: 0.1530
Epoch: 1, Batch: 800/1250, Loss: 0.4431
Epoch: 1, Batch: 900/1250, Loss: 0.1834
Epoch: 1, Batch: 1000/1250, Loss: 0.5279
Epoch: 1, Batch: 1100/1250, Loss: 0.0600
Epoch: 1, Batch: 1200/1250, Loss: 0.5920
Epoch: 1, Average Loss: 0.0970
Epoch: 2, Batch: 100/1250, Loss: 0.1403
Epoch: 2, Batch: 200/1250, Loss: 0.0956
Epoch: 2, Batch: 300/1250, Loss: 0.2718
Epoch: 2, Batch: 400/1250, Loss: 0.3578
Epoch: 2, Batch: 500/1250, Loss: 0.4648
Epoch: 2, Batch: 600/1250, Loss: 0.1099
Epoch: 2, Batch: 700/1250, Loss: 0.1014
Epoch: 2, Batch: 800/1250, Loss: 0.2920
Epoch: 2, Batch: 900/1250, Loss: 0.0800
Epoch: 2, Batch: 1000/1250, Loss: 0.2682
Epoch: 2, Batch: 1100/1250, Loss: 0.0202
Epoch: 2, Batch: 1200/1250, Loss: 0.2810
Epo

In [32]:
model.eval()

total_val_loss = 0.0
num_val_batches = len(val_loader)

for i, batch in enumerate(val_loader, 1):
    # Unpack the batch
    input_ids = batch[0].to(device)
    attention_mask = batch[1].to(device)
    labels = batch[2].to(device)

    # Forward pass
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

    total_val_loss += loss.item()

    print(f"Batch: {i}/{len(val_loader)}, Loss: {loss.item():.4f}")

average_val_loss = total_val_loss / num_val_batches

print(f"Average Validation Loss: {average_val_loss:.4f}")

Batch: 1/125, Loss: 0.0645
Batch: 2/125, Loss: 0.0914
Batch: 3/125, Loss: 0.0755
Batch: 4/125, Loss: 0.0421
Batch: 5/125, Loss: 0.0466
Batch: 6/125, Loss: 0.0855
Batch: 7/125, Loss: 0.0827
Batch: 8/125, Loss: 0.1541
Batch: 9/125, Loss: 0.1214
Batch: 10/125, Loss: 0.0374
Batch: 11/125, Loss: 0.0199
Batch: 12/125, Loss: 0.0226
Batch: 13/125, Loss: 0.0838
Batch: 14/125, Loss: 0.0449
Batch: 15/125, Loss: 0.0021
Batch: 16/125, Loss: 0.2519
Batch: 17/125, Loss: 0.0034
Batch: 18/125, Loss: 0.0201
Batch: 19/125, Loss: 0.0208
Batch: 20/125, Loss: 0.0852
Batch: 21/125, Loss: 0.0066
Batch: 22/125, Loss: 0.0504
Batch: 23/125, Loss: 0.0028
Batch: 24/125, Loss: 0.0475
Batch: 25/125, Loss: 0.0395
Batch: 26/125, Loss: 0.0467
Batch: 27/125, Loss: 0.0062
Batch: 28/125, Loss: 0.0209
Batch: 29/125, Loss: 0.0228
Batch: 30/125, Loss: 0.0201
Batch: 31/125, Loss: 0.0095
Batch: 32/125, Loss: 0.0374
Batch: 33/125, Loss: 0.0422
Batch: 34/125, Loss: 0.0388
Batch: 35/125, Loss: 0.0423
Batch: 36/125, Loss: 0.1031
B

In [33]:
# evaluate the model randomly

def evaluateRandomly(model, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = model.generate(**tokenizer(pair[0], return_tensors="pt", padding=True))
        output_sentence = tokenizer.decode(output_words[0], skip_special_tokens=True)
        print('<', output_sentence)
        print('')
evaluateRandomly(model)


> tom has red hair and freckles .
= tom hat rotes haar und sommersprossen .
< tom hat rotes hair und freckles.

> i had hoped we might become friends .
= ich hatte gehofft wir konnten freunde werden .
< ich habe mich erhofft wir konnen freund gehen.

> i want to consult you about something .
= ich hatte gerne deinen rat in einer sache .
< ich mochte dich zu etwas beruhigen.

> tom says that he s tired .
= tom sagt dass er mude ist .
< tom sagt er ist mude.

> i ve never found that to be necessary .
= ich habe das nie fur notig befunden .
< ich habe das nie für notwendig gefunden.

> i think he s a great writer .
= ich finde er ist ein toller schriftsteller .
< ich stimme zu, er ist ein grossartiger Schriftstellerin.

> tom always made me laugh .
= tom brachte mich immer zum lachen .
< tom hat mich immer zum lacheln gekommen.

> tom has few friends .
= tom hat wenige freunde .
< tom hat noch ein sein freund.

> he denied the accusation .
= er verneinte die anklage .
< er bestritt die An

# Credits

This problem set is based upon an official PyTorch [tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html). Many thanks to PyTorch, [Sean Robertson](https://github.com/spro/practical-pytorch) and  [Florian Nachtigall](https://github.com/FlorianNachtigall).

Be cautious with looking in the original notebook for answers. Many details have been changed and you won't be able to copy-and-paste solutions.
